In [ ]:
import findspark
import folium

findspark.init()

from pyspark import sql, SparkConf, SparkContext

coordinates = (20.593684, 78.96288)
conf = SparkConf().setAppName("Read_CSV")
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)
m = folium.Map(location=coordinates, zoom_start=4)

dfCities = sqlContext.read.csv("Steden.csv", header=True)
dfPollution = sqlContext.read.csv("Luchtvervuiling.csv", header=True)

In [ ]:
from folium.plugins import MarkerCluster
from ipywidgets import interact

schemes = ["so2", "no2"]
dfCities = dfCities.select("lat", "lng", "city")
dfPollution = dfPollution.select("location", "so2", "no2")
c = '#3186cc'
joinDF = dfPollution.join(dfCities, dfPollution.location == dfCities.city, how='right')
joinDF = joinDF.filter("no2 != ''").filter("lat != ''").filter("lng != ''").dropDuplicates(['city']).collect()

@interact(scheme=schemes)
def get_pollution(scheme):
    index = 0
    for line in joinDF:
        if scheme == 'so2':
            pollution = joinDF[index][1]
        else:
            pollution = joinDF[index][2]
        hasData = (pollution != 'NA') and (pollution is not None)
        if hasData and (float(pollution) > 18):
            c = '#ff0000'
        elif hasData and (float(pollution) < 10):
            c = '#7CFC00'
        else:
            c = '#ffd27f'
        folium.CircleMarker(
            location=[joinDF[index][3], joinDF[index][4]],
            radius=4,
            popup=pollution,
            color=c,
            fill=True,
            fill_color='#3186cc'
        ).add_to(m)
        index = index + 1
    return m